<a href="https://colab.research.google.com/github/Tabook22/AI/blob/main/Good_WSD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy tqdm scikit-learn transformers torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
ERROR: Operation cancelled by user


In [ ]:
import json
import numpy as np
from tqdm import tqdm
from collections import defaultdict
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, AutoModel
import torch
import time

In [ ]:
# Load CAMeLBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("CAMeL-Lab/bert-base-arabic-camelbert-msa")
model = AutoModel.from_pretrained("CAMeL-Lab/bert-base-arabic-camelbert-msa")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/468 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/305k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

In [ ]:
# Function to load JSON data
def load_json(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return json.load(f)

In [ ]:
# Prepare labeled data for training and evaluation
def prepare_labeled_data(data, wsd_dict):
    labeled_data = []
    for entry in data:
        word = entry['word']
        lemma_id = entry['lemma_id']
        correct_gloss_id = entry.get('gloss_id')
        for sense in wsd_dict[lemma_id]:
            labeled_data.append({
                'context_id': entry['context_id'],
                'context': entry['context'],
                'word': word,
                'lemma_id': lemma_id,
                'gloss_id': sense['gloss_id'],
                'gloss': sense['gloss'],
                'label': 1 if sense['gloss_id'] == correct_gloss_id else 0
            })
    return labeled_data

In [ ]:
# Extract or fetch cached embeddings for a given text
embedding_cache = defaultdict(lambda: None)

In [ ]:
def encode_text(text):
    tokens = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    return tokens

def extract_embeddings(text):
    tokens = encode_text(text)
    with torch.no_grad():
        output = model(**tokens)
    return output.last_hidden_state.mean(dim=1).squeeze().numpy()

def extract_embeddings_with_cache(text):
    if embedding_cache[text] is None:
        embedding_cache[text] = extract_embeddings(text)
    return embedding_cache[text]

In [ ]:
# Prepare features with progress monitoring every minute
def prepare_features(data, log_interval=60):
    features = []
    start_time = time.time()
    last_update_time = start_time
    for i, entry in enumerate(tqdm(data, desc="Extracting Features", unit="pair")):
        context_emb = extract_embeddings_with_cache(entry['context'])
        gloss_emb = extract_embeddings_with_cache(entry['gloss'])
        label = entry['label']
        features.append((context_emb, gloss_emb, label))

        current_time = time.time()
        if current_time - last_update_time >= log_interval:
            elapsed = current_time - start_time
            progress = (i + 1) / len(data)
            remaining = elapsed / progress - elapsed
            tqdm.write(
                f"Progress: {progress * 100:.2f}% - Elapsed: {elapsed:.2f}s - Remaining: {remaining:.2f}s"
            )
            last_update_time = current_time
    return features

In [ ]:
# Concatenate features for classifier input
def concatenate_features(features):
    X, y = [], []
    for context_emb, gloss_emb, label in features:
        X.append(np.concatenate((context_emb, gloss_emb)))
        y.append(label)
    return np.array(X), np.array(y)

In [ ]:
# Load all datasets
train_data = load_json('/content/drive/MyDrive/WSD/Data/train.json') #The training dataset used to train the WSD model.
dev_data = load_json('/content/drive/MyDrive/WSD/Data/dev.json')#The development or validation dataset used for model tuning and intermediate evaluations.
#test_data = load_json('/content/drive/MyDrive/WSD/Data/test.json')#The test dataset used for final evaluation of the trained model.
test_data = load_json('/content/drive/MyDrive/WSD/Data/test2.json')#The test dataset used for final evaluation of the trained model.
wsd_dict_raw = load_json('/content/drive/MyDrive/WSD/Data/WSD_dict.json')#The dictionary containing the mapping between lemma IDs and their corresponding glosses.

NameError: name 'load_json' is not defined

In [ ]:
# Prepare WSD dictionary
wsd_dict = defaultdict(list)
for entry in wsd_dict_raw:
    wsd_dict[entry['lemma_id']].append({
        'gloss_id': entry['gloss_id'],
        'gloss': entry['gloss']
    })

In [ ]:
# Prepare labeled data
train_labeled = prepare_labeled_data(train_data, wsd_dict)
dev_labeled = prepare_labeled_data(dev_data, wsd_dict)

In [ ]:
# Extract features for training and development
train_features = prepare_features(train_labeled)
dev_features = prepare_features(dev_labeled)

Extracting Features:   9%|▉         | 10258/115424 [01:00<29:53, 58.63pair/s]

Progress: 8.88% - Elapsed: 60.02s - Remaining: 616.16s


Extracting Features:  12%|█▏        | 13474/115424 [02:00<49:37, 34.24pair/s]

Progress: 11.67% - Elapsed: 120.08s - Remaining: 908.85s


Extracting Features:  16%|█▌        | 18183/115424 [03:00<10:22, 156.25pair/s]

...

Extracting Features:  93%|█████████▎| 107417/115424 [23:01<01:12, 110.76pair/s]

Progress: 93.06% - Elapsed: 1381.60s - Remaining: 102.99s


Extracting Features:  99%|█████████▉| 114386/115424 [24:01<00:08, 121.24pair/s]

Progress: 99.08% - Elapsed: 1441.63s - Remaining: 13.37s


Extracting Features: 100%|██████████| 14834/14834 [00:57<00:00, 257.84pair/s]


In [ ]:
# Prepare feature matrices
X_train, y_train = concatenate_features(train_features)
X_dev, y_dev = concatenate_features(dev_features)

In [ ]:
# Train a logistic regression classifier with class weight adjustment
class_weights = {0: 1, 1: 3}
clf = LogisticRegression(max_iter=1000, class_weight=class_weights)
clf.fit(X_train, y_train)

LogisticRegression(class_weight={0: 1, 1: 3}, max_iter=1000)

In [ ]:
# Evaluate the model
y_pred = clf.predict(X_dev)
print(classification_report(y_dev, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.82      0.87     12033
           1       0.47      0.68      0.56      2801

    accuracy                           0.79     14834
   macro avg       0.69      0.75      0.71     14834
weighted avg       0.83      0.79      0.81     14834



In [ ]:
# Predict gloss for each word in the test data
def predict_gloss(word_context, glosses):
    context_emb = extract_embeddings_with_cache(word_context)
    scores = []
    for gloss in glosses:
        gloss_emb = extract_embeddings_with_cache(gloss['gloss'])
        concatenated = np.concatenate((context_emb, gloss_emb))
        score = clf.predict_proba([concatenated])[0][1]
        scores.append((gloss['gloss_id'], score))
    return max(scores, key=lambda x: x[1])[0]

In [ ]:
def evaluate_on_test(test_data, wsd_dict):
    correct = 0
    total = len(test_data)
    for entry in test_data:
        word_context = entry['context']
        lemma_id = entry['lemma_id']
        possible_glosses = wsd_dict.get(lemma_id, [])
        if not possible_glosses:
            continue
        predicted_gloss_id = predict_gloss(word_context, possible_glosses)
        if predicted_gloss_id == entry.get('gloss_id'):
            correct += 1
    accuracy = correct / total
    return accuracy

In [ ]:
# Evaluate on the test data
test_accuracy = evaluate_on_test(test_data, wsd_dict)
print(f"Test Accuracy: {test_accuracy:.4f}")

Test Accuracy: 0.6815


**In the code bellow the predict_gloss function takes two parameters:**


**word_context:** This represents the context in which a word appears. In the evaluate_on_test_with_progress_and_debug function, the word_context is obtained from each entry in the test_data. Specifically, it is retrieved using word_context = entry['context'].

This suggests that the test_data is a list or iterable of dictionaries, where each dictionary represents an entry or instance in the test set. The 'context' key in each dictionary holds the actual context or surrounding text in which the word appears.

**glosses:** This is a list of possible meanings or definitions (glosses) for the word being disambiguated.

In the evaluate_on_test_with_progress_and_debug function, the possible_glosses are obtained from the wsd_dict dictionary using the lemma_id of each entry.

The line **possible_glosses = wsd_dict.get(lemma_id, [])** retrieves the list of glosses associated with the lemma_id from the wsd_dict dictionary.

If no glosses are found for a particular lemma_id, an **empty list []** is returned as a default value.


The **wsd_dict **dictionary is likely loaded or constructed from a separate file or data source that contains the mapping between lemma IDs and their corresponding glosses. The structure of wsd_dict is expected to be as follows:

In [ ]:
# Predict gloss for each word context and add debug prints
def predict_gloss(word_context, glosses):
    context_emb = extract_embeddings_with_cache(word_context)
    scores = []
    for gloss in glosses:
        gloss_emb = extract_embeddings_with_cache(gloss['gloss'])
        concatenated = np.concatenate((context_emb, gloss_emb))
        score = clf.predict_proba([concatenated])[0][1]
        scores.append((gloss['gloss_id'], score))
    predicted_gloss_id, _ = max(scores, key=lambda x: x[1])
    return predicted_gloss_id

# Function to evaluate the model on the test data with debug prints
def evaluate_on_test_with_progress_and_debug(test_data, wsd_dict):
    correct = 0
    total = len(test_data)
    start_time = time.time()
    last_update_time = start_time

    for i, entry in enumerate(tqdm(test_data, desc="Evaluating", unit="pair")):
        word_context = entry['context']
        lemma_id = entry['lemma_id']
        possible_glosses = wsd_dict.get(lemma_id, [])
        if not possible_glosses:
            continue
        predicted_gloss_id = predict_gloss(word_context, possible_glosses)
        correct_gloss_id = entry.get('gloss_id')
        if predicted_gloss_id == correct_gloss_id:
            correct += 1
        else:
            print(f"Context: {word_context}, Predicted Gloss: {predicted_gloss_id}, Correct Gloss: {correct_gloss_id}")

        current_time = time.time()
        if current_time - last_update_time >= 60:
            elapsed = current_time - start_time
            progress = (i + 1) / total
            remaining = elapsed / progress - elapsed
            tqdm.write(
                f"Progress: {progress * 100:.2f}% - Elapsed: {elapsed:.2f}s - Remaining: {remaining:.2f}s"
            )
            last_update_time = current_time

    accuracy = correct / total if total else 0
    elapsed_time = time.time() - start_time
    print(f"Evaluation Completed - Elapsed Time: {elapsed_time:.2f}s")
    return accuracy

# Evaluate on the test data with debug prints
test_accuracy = evaluate_on_test_with_progress_and_debug(test_data, wsd_dict)
print(f"Test Accuracy: {test_accuracy:.4f}")

Evaluating:   2%|▏         | 84/4496 [00:00<00:05, 836.22pair/s]

Context: حالة موقع تحريج كفردبيل- ريف جبلة ", Predicted Gloss: gloss.15993, Correct Gloss: gloss.15992
Context: وليس للقانون أثر رجعي., Predicted Gloss: gloss.14893, Correct Gloss: gloss.14888
Context: بحسب رئيس جمعية مزارعي فزان " عثمان الطاهر" حيث وصل إنتاج ليبيا من القمح والشعير خلال العام الماضي إلى (100 ) ألف طن،, Predicted Gloss: gloss.4777, Correct Gloss: gloss.4775
Context: - الدرجة مكونة من مرتبتين (احاد وعشرات) مرتبة العشرات تاخذ القيم (6,7,8,9) فمرتبة (6) ظهرت مرة واحدة لان كل درجات مرتبة عشرات (6) اكبر من (65) ،, Predicted Gloss: gloss.5874, Correct Gloss: gloss.5873
Context: حيث يذكر في الباب الأول (حقوق الشعب) نصوصاً قانونية تتعلق بالجنسية العراقية،, Predicted Gloss: gloss.1599, Correct Gloss: gloss.1590
Context: وأضاف ان الشركة عندما بدأت أعمالها كان اهتمامها يقتصر على الأدوات المنزلية الكهربائية من برادات وغسالات وأفران ومكانس كهربائية., Predicted Gloss: gloss.2782, Correct Gloss: gloss.2784
Context: والتي هي من أهم مبادئ حقوق الانسان في وقتنا الحاضر واعادة تعمير ماخربه

Evaluating:   6%|▌         | 264/4496 [00:00<00:05, 829.19pair/s]


Context: مما يُحدث التهابًا بالكبد وضعف قدرته على القيام بوظائفه., Predicted Gloss: gloss.1237, Correct Gloss: gloss.1235
Context: مقارنةً بالخلايا الكيراتينية التي تُشكل جُريبات الشعر،, Predicted Gloss: gloss.3483, Correct Gloss: gloss.3481
Context: بالنسبة لانتشار النوع الفرعي الرابع الوردية العينية ( (OR فهو غير معروف بسبب عدم خصوصية التظاهرات السريرية, وقد ارتبطت التظاهرات السريرية العينية مع التظاهرات الجلدية في الوردية في 33 ٪ من 100 مريض وردية جلدية في اليونان, وسجل حدوث الوردية العينية في المملكة المتحدة إلى 1., Predicted Gloss: gloss.2639, Correct Gloss: gloss.2636
Context: وعرفه (جوفمان) على أنه "كل نشاط الفرد الذي يحدث خلال فترة ما،, Predicted Gloss: gloss.812, Correct Gloss: gloss.814
Context: 2-	التاريخ يقوي اعتزاز المواطن بأمته ووطنه وبشهداء هذه الأمة والابطال المدافعين عنها., Predicted Gloss: gloss.7102, Correct Gloss: gloss.7100
Context: شبكة توليدية وشبكة تمييزية،, Predicted Gloss: gloss.3278, Correct Gloss: gloss.3271
Context: وعـدت إلـى الأهـل أخيـرًاً ” قـال نيكـول

Evaluating:  10%|█         | 459/4496 [00:00<00:04, 914.24pair/s]

...

Context: لكن ومن خلال استخدام البيانات المصطنعة المدربة على مجموعات بيانات أكبر بكثير،, Predicted Gloss: gloss.3013, Correct Gloss: gloss.3012
Context: ولذلك فقد اتفق على أن تستخدم في حسابها وحدة خاصة هي السنة الضوئية Light Year وهي المسافة التي يقطعها الضوء "وسرعته 300 ألف كيلو متر في الثانية" في سنة كاملة،, Predicted Gloss: gloss.11242, Correct Gloss: gloss.11246
Context: ‮ ‬حتّى الحروب التّحريريّة من الاستعمار،, Predicted Gloss: gloss.8471, Correct Gloss: gloss.8472
Context: هـو “شـعلة الحطـب” برنـار،, Predicted Gloss: gloss.10345, Correct Gloss: gloss.10344
Context: الدستور حسبما ورد في تقرير اللجنة الوطنية للإحصاء., Predicted Gloss: gloss.8858, Correct Gloss: gloss.8857
Context: التقويم يعني بيان السنة والشهر واليوم وضبطها ., Predicted Gloss: gloss.8170, Correct Gloss: gloss.8169
Context: ومـن يريـد أن يطالـع مـا يكتـب عـن مشـاهد القتـال المتواصـل., Predicted Gloss: gloss.1864, Correct Gloss: gloss.1861
Context: لا يمكن منح الكائنات الحية حريات طبيعية مطلقة دون ضوابط, Predicted Gl

Evaluating:  99%|█████████▉| 4442/4496 [00:10<00:00, 279.47pair/s]


Context: قـوات بقيـادة قائـد جيـش منطقـة قـازان سـتولنيك إيفـان موراشـكين الـذي أعـدم مـن قبـض عليهـم،, Predicted Gloss: gloss.9010, Correct Gloss: gloss.9009
Context: كان يتكلم بالرأى فيخطئ ويصيب ., Predicted Gloss: gloss.432, Correct Gloss: gloss.431
Context: طريـق الحريـر ., Predicted Gloss: gloss.14707, Correct Gloss: gloss.14701
Context: كبلاد الهند التي تضمّ مجموعةً من العادات الغريبة التي تسجّلها أعين الكاميرات،, Predicted Gloss: gloss.2774, Correct Gloss: gloss.2772
Context: الهدف هو إنشاء منتجات وعمليات وسياسات – طرق جديدة للحياة – تتكيف بشكل جيد مع الحياة على الأرض على المدى الطويل., Predicted Gloss: gloss.5449, Correct Gloss: gloss.5448
Context: بنغازي 27 فبراير 2022 م ( وال ) - تتجه أنظار العالم هذه الأيام نحو الأزمة التي اندلعت بين روسيا وأوكرانيا والتوترات الحاصلة في المنطقة إثر الاجتياح الروسي للأراضي الأوكرانية والتي تنذر بوقوع أزمات اقتصادية في العالم أجمع ،, Predicted Gloss: gloss.2983, Correct Gloss: gloss.2982
Context: 470), وهذا الفهم في كثير من الأحيان لا يشمل ال

Evaluating: 100%|██████████| 4496/4496 [00:10<00:00, 439.19pair/s]

Context: وتطال أعناقهم إلى تواريخ الأمم الخالية،, Predicted Gloss: gloss.10992, Correct Gloss: gloss.10988
Context: وفــي مطلــع الثلاثينــات نشــرت أول روايــة شيشــانية بقلــم ســعيد بــادروف., Predicted Gloss: gloss.1599, Correct Gloss: gloss.1590
Context: ذلك إلى أن يبلغ الملك سن الرشد،, Predicted Gloss: gloss.9745, Correct Gloss: gloss.9743
Context: وباعتباره وسيلة الأنظمة التعليمية لتشكيل الخبرات التربوية؛, Predicted Gloss: gloss.10674, Correct Gloss: gloss.10664
Context: وعرفه (أبراهامز) عندما قدم النظرية البلاغية للفولكلور:, Predicted Gloss: gloss.5543, Correct Gloss: gloss.5538
Context: وهي صفات للاستجابة الثنائية والتي توضح بصورة ادق بالمثال التالي ., Predicted Gloss: gloss.5332, Correct Gloss: gloss.5328
Context: ان متغير بيانات الاستجابه الثنائية (binary response) يمتلك حالتين فقط مثل وجود او عدم الصفة ،, Predicted Gloss: gloss.5332, Correct Gloss: gloss.5328
Evaluation Completed - Elapsed Time: 10.25s
Test Accuracy: 0.6815
